Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [4]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
from random import random, choice, randint
from copy import deepcopy,copy
from dataclasses import dataclass

## The *Nim* and *Nimply* classes

In [5]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [6]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


In [ ]:
TOURNAMENT_SIZE = 2
NUM_POPULATIONS = 100
MUTATION_PROBABILITY = .15 
DIM_CHROM = 15
OPERATIONS = ["+","-","*","/","and","%","or","not","xor"]

In [ ]:
@dataclass
class Gene:
    fitness: tuple
    genotype: list[str]
    self_expression : int

    def get_expression(self, position_tuple) ->int:
        self_expression = self.recursive_decoder(len(self.expr)-1, position_tuple)
        return self_expression
    
    def mutate(self, state: Nim) -> None:
        offspring = copy(self)
        pos = randint(0, len(self.genotype)-1)
        if len(offspring.genotype[pos]) == 1:
            offspring.genotype[pos] = str(randint(0,len(state._rows)-1))
        else:
            offspring[pos][0] = OPERATIONS[randint(0,len(OPERATIONS) - 1)]
        offspring.fitness = None
        return offspring

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion



def one_cut_xover(ind1: Gene, ind2: Gene) -> Gene:
    cut_point = randint(0, ind1.genotype-1)
    offspring = Gene(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == len(ind1.genotype)
    return offspring

def recursive_decoder(self : str, index, position_tuple):
        if len(self[index] == 1):    
            return position_tuple[self.expr[index]]
        else:                                       #checks the operator and calls recursively
            operator = self.expr[index][0]
            if operator=="+":
                op1 = self.recursive_decoder(self.expr[index][1], position_tuple)
                op2 = self.recursive_decoder(self.expr[index][2], position_tuple)
                return op1+op2
            if operator=="*":
                op1 = self.recursive_decoder(self.expr[index][1], position_tuple)
                op2 = self.recursive_decoder(self.expr[index][2], position_tuple)
                return op1*op2
            if operator=="-":
                op1 = self.recursive_decoder(self.expr[index][1], position_tuple)
                op2 = self.recursive_decoder(self.expr[index][2], position_tuple)
                return abs(op1-op2)
            if operator=="%":
                op1 = self.recursive_decoder(self.expr[index][1], position_tuple)
                op2 = self.recursive_decoder(self.expr[index][2], position_tuple)
                return op1%op2
            if operator=="and":
                op1 = self.recursive_decoder(self.expr[index][1], position_tuple)
                op2 = self.recursive_decoder(self.expr[index][2], position_tuple)
                return op1&op2
            if operator=="or":
                op1 = self.recursive_decoder(self.expr[index][1], position_tuple)
                op2 = self.recursive_decoder(self.expr[index][2], position_tuple)
                return op1|op2
            if operator=="xor":
                op1 = self.recursive_decoder(self.expr[index][1], position_tuple)
                op2 = self.recursive_decoder(self.expr[index][2], position_tuple)
                return op1^op2


    
def fitness_function(self : Gene, gameTree) -> int:
    fit_container = [0]
    gameTree.set_NP(self)
    gameTree.traverse_tree(gameTree.nodes[0], None, fit_container)
    fit = fit_container[0]
    return fit

In [ ]:
class TreeNode:
    def __init__(self, dots, is_visited=False):
        self.dots = dots  
        self.position_type = False  
        self.father = None  
        self.children = []

    def add_father(self, father_node):
        self.father = father_node

    def add_child(self, child_node):
        self.children.append(child_node)
    
    def check_rules(self):
        flag = 0
        if self.position_type == True: #no P after a P
            for child in self.children:
                if child.position_type == True:
                    flag = 1
                    break
        if self.position_type == False: #at least one P after a N
            flag = 1
            for child in self.children:
                if child.position_type == True:
                    flag = 0
                    break
        if sum(self.dots)==0:       #terminal step should be P
            if self.position_type == False:
                flag = 1
        return flag


class Tree:
    def __init__(self, dots):
        self.nodes = []
        starting_node = self.add_node(dots)

        def createNode(current_node, current_tuple):
            for i, value in enumerate(current_tuple):
                for new_value in range(value, -1, -1):
                    if new_value != value:
                        new_tuple = current_tuple[:i] + (new_value,) + current_tuple[i + 1:]
                        new_node = self.add_node(new_tuple)
                        self.add_edge(current_node, new_node)
                        createNode(new_node, new_tuple)

        createNode(starting_node, dots)

    def add_node(self, dots, is_visited=False):
        new_node = TreeNode(dots, is_visited)
        self.nodes.append(new_node)
        return new_node

    def add_edge(self, father, child):
        father.add_child(child)
        child.add_father(father)
    
    def traverse_tree(self, node, chromosome : Gene = None, score_container=None):
        if score_container is None:
            score_container = [0]
        
        # traversing to set NP #############################
        if chromosome is not None:
            current_score = chromosome.get_expression(node.dots)
            if current_score == 0:
                node.position_type = True
            else:
                node.position_type = False
        ####################################################

        
        # traversing to get fitness ########################
        else:
            current_score = node.check_rules()
            score_container[0] += current_score
        ####################################################

        for child in node.children:
            self.traverse_tree(child, chromosome, score_container)
        

    def set_NP(self, chromosome):
        self.traverse_tree(self.nodes[0], chromosome)

In [ ]:
def creation(start, end) -> str:
    add1 = randint(start-1,end-1)
    return str(add1)

def create_population(state : Nim):
    rows = len(state._rows)
    operations = copy(OPERATIONS)
    nodi = list[str]
    popolazione = list[Gene]
    for i in range (rows):
        nodi.append(str(i))
        operations.append(str(i))
    for i in range(NUM_POPULATIONS):
        cromo = list[str]
        for k in range(0,DIM_CHROM):
            if k < 2:
                op = creation(1,len(nodi))
            else:
                op = creation(1,len(operations))
                if int(op) < len(OPERATIONS):
                    add1 = creation(1,len(cromo))
                    add2 = creation(1,len(cromo))
                    while add1 == str(k):            
                        add1 = creation(1,len(cromo))
                    while add2 == str(k) and add2 == add1:
                        add2 = creation(1,len(cromo))
                    op = op + add1 + add2
            cromo.append(op)
        if sum(len(stringa) for stringa in cromo) == len(cromo):
            i -= 1
        else:
            gene = Gene(genotype = cromo, fitness = None, self_expression = None)
            popolazione.append(gene)

def run(pop: list[Gene], state : Nim) -> list[Gene]:
    for k in range(0,len(pop),4):
        p1 = randint(0,3)
        p2 = randint(0,3)
        while(p1 == p2):
            p1 = randint(0,3)
        son = generazione(pop[k+p1],pop[k+p2],state)
        gene = min(pop[k:k+4],key=lambda i: i.fitness)
        if son.fitness > gene.fitness:
            counter = 0
            while gene != pop[k+counter]: counter += 1
            pop[k+counter] =  son
    return pop
    
        

def generazione(parent1 : Gene, parent2 : Gene, state: Nim) -> Gene:
    new = one_cut_xover(parent1,parent2)
    if(random < MUTATION_PROBABILITY):
        new = mutate(parent1,parent2,state)
    new.fitness = fitness_function()
    return new

## Sample (and silly) startegies 

In [7]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [8]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [9]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [10]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


# Evolutionary Strategy
## Chromosome and Tree classes

## Oversimplified match

In [18]:
nim = Nim(3)
gameTree = Tree(nim.rows)
ch1 = Chromosome((0, 1, ("xor",0,1), 2, ("xor", 2, 3)))
ch2 = Chromosome((0, 1, ("and",0,1), 2, ("or", 2, 3)))

print(f"fitness 1 is {ch1.fitness_function(gameTree)}")
print(f"fitness 2 is {ch2.fitness_function(gameTree)}")
logging.info(f"{nim}")

#player = 0
#while nim:
#    ply = strategy[player](nim)
#    logging.info(f"ply: player {player} plays {ply}")
#    nim.nimming(ply)
#    logging.info(f"status: {nim}")
#    player = 1 - player
#logging.info(f"status: Player {player} won!")


fitness 1 is 0
fitness 2 is 2648
